In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve
from motion_integration import (integrate_motion, time_step, eps_picker, find_bond_forces, 
                                repulsive_force, evaluate_motion_equations)
from force_test import assemble_quad_matrix
from resistance_matrix_test import generate_resistance_matrices

In [2]:
def get_tstep_distribution(expt_name):
    data_dir = '/Users/andrewwork/thesis/reg_stokeslets/3d/data/'
    data = np.load(data_dir + expt_name + '.npz')
    diffs = data['t'][1:] - data['t'][:-1]
    return np.bincount(np.round(-np.log2(diffs / .002)).astype('int'))

### Time steps in the experiments with stiff bonds

In [3]:
t_steps = get_tstep_distribution('bd_expt089')
print(t_steps)

[183  14  38  91 161 657 423 223  22  25  41  10]


In [4]:
t_steps2 = get_tstep_distribution('bd_expt092')
print(t_steps2)

[170  17  70 158 226 502 262  43   1   2]


### Forces  and torques in stable configuration

In [5]:
n_nodes = 8
a, b = 1.5, 0.5
domain  = 'wall'
eps = eps_picker(n_nodes, a, b)
precompute_array = None
proc = 1
shear_vec = True
kappa = 25000.
lam = 0.1

In [6]:
data = np.load('data/bd_expt092.npz')
ii = np.argwhere((data['t'] >= 0.35) * (data['t'] <= 0.36))[0][0]
distance = data['x'][ii]
center = np.array([data['x'][ii], data['y'][ii], data['z'][ii]])
rmat = data['r_matrices'][..., ii]
receptors = data['receptors']
bonds = data['bond_array'][..., ii]
print(data['t'][ii])
print(data['t'][ii-1])

0.35049999999999953
0.3499999999999996


In [7]:
data['bond_array'][..., ii-2]

array([[ 5.76000000e+02, -1.00000000e-01,  3.18235818e+01]])

In [8]:
theta = np.arctan2(rmat[2, 0], rmat[1, 0])
phi = np.arccos(rmat[0, 0])

In [9]:
result = generate_resistance_matrices(
    eps, n_nodes, a, b, domain, distance, theta, 
    phi, shear_vec, proc, precompute_array
)

t_matrix, p_matrix, pt_matrix, r_matrix, shear_f, shear_t = result
res_matrix = np.block([[t_matrix, p_matrix], [pt_matrix, r_matrix]])

Assembling quadrature matrix for eps = 0.1261902586321847, nodes = 8
Solving for forces for eps = 0.1261902586321847, nodes = 8
Finished eps=0.1261902586321847, n_nodes=8


In [10]:
bd_forces, bd_torques = find_bond_forces(receptors, bonds, center, rmat, kappa, lam, one_side=True)
point, rep_force = repulsive_force(center[0], rmat[:, 0])
point = np.array([point[0], point[1] + center[1], point[2] + center[2]])
rep_forces = np.array([rep_force, 0, 0])
rep_torques = np.cross([point[0] - center[0], point[1], point[2]],
                        [rep_force, 0, 0])
forces, torques = rep_forces + bd_forces, rep_torques + bd_torques

In [11]:
print(bd_forces)
print(bd_torques)

[-4.08577535e+02 -9.46426842e-02 -4.81242980e+01]
[ 4.63569260e-02  3.71075454e+02 -1.12334055e+00]


In [12]:
gen_forces = np.block([[shear_f - forces.reshape((3, 1))], 
                       [shear_t - torques.reshape((3, 1))]])
gen_vels = np.linalg.solve(res_matrix, -gen_forces)
trans_vels, rot_vels = np.squeeze(gen_vels[:3]), np.squeeze(gen_vels[3:])

In [13]:
print(gen_forces.T)
print(trans_vels)
print(rot_vels)

[[-3.94296613e+02  6.79025948e-02 -4.53004149e-01 -7.41672854e-02
  -2.63466059e+04 -7.98016382e+01]]
[ 1.49143017e+02  1.48607645e-01 -7.07301683e+01]
[  0.25036164 184.50581791   0.52867516]


In [14]:
print(np.dot(res_matrix, gen_vels)[3:] + shear_t 
      + np.cross((np.dot(res_matrix, gen_vels)[:3] + shear_f).T, point - center).T)

print(bd_torques + np.cross(bd_forces, point - center))

[[-4.96448348e-03]
 [ 2.67634219e+04]
 [ 7.85839817e+01]]
[-4.96448348e-03  8.44488721e+01 -1.23930982e-01]


In [15]:
point - center

array([-1.1731818 , -0.00271783, -0.83970631])

In [16]:
print(forces - shear_f.T)
print(torques - shear_t.T)

[[ 3.94296613e+02 -6.79025948e-02  4.53004149e-01]]
[[7.41672854e-02 2.63466059e+04 7.98016382e+01]]


In [17]:
print(t_matrix)

[[ 3.10921257e+03 -3.69812207e-02 -1.14258103e+01]
 [-3.69812207e-02  8.17973784e+01  6.29858127e-02]
 [-1.14258103e+01  6.29858127e-02  1.01257428e+02]]


In [18]:
shear_t

array([[-2.78103594e-02],
       [ 2.19202075e+01],
       [-1.12243341e-02]])

In [19]:
np.dot(res_matrix, gen_vels) + gen_forces

array([[-2.38742359e-11],
       [ 2.08166817e-16],
       [-9.59232693e-13],
       [-2.30371278e-15],
       [-6.54836185e-11],
       [ 4.83169060e-13]])